In [17]:
import fiddle as fdl
from fiddle.experimental import auto_config
from fiddle.codegen import codegen
from fiddle import printing
from rich import inspect, print

from datasets.dataset import Dataset
from graphs.graph import Graph
from models.model import Model
from models.tasks.task import Task
from training.trainer import Trainer

@auto_config.auto_config
def make_trainer():
    """Creates a trainer.
    """
    dataset = Dataset(name="era", size=1000)
    # dataset should be of type Dataset, not str
    #dataset = 'ooops'
    graph = Graph(dataset=dataset, nodes=15)

    model = Model(graph=graph, hidden_dim=64)
    task = Task(model=model, task_type="classifier")
    trainer = Trainer(task=task, dataset=dataset, epochs=5)
    return trainer


cfg = make_trainer.as_buildable()

In [18]:
print(cfg)

<Config[Trainer(
  task=<Config[Task(
    model=<Config[Model(
      graph=<Config[Graph(dataset=<Config[Dataset(name='era', size=1000)]>, nodes=15)]>,
      hidden_dim=64)]>,
    task_type='classifier')]>,
  dataset=<Config[Dataset(name='era', size=1000)]>,
  epochs=5)]>

In [19]:
print(printing.as_str_flattened(cfg))

task.model.graph.dataset.name: str = 'era'
task.model.graph.dataset.size: int = 1000
task.model.graph.nodes: int = 15
task.model.hidden_dim: int = 64
task.task_type: str = 'classifier'
dataset.name: str = 'era'
dataset.size: int = 1000
epochs: int = 5

In [20]:
generated = codegen.codegen_dot_syntax(cfg)
print("\n".join(generated.lines()))

from datasets import dataset
import fiddle as fdl
from graphs import graph
from models import model
from models.tasks import task
from training import trainer


def build_config():
  shared_dataset = fdl.Config(dataset.Dataset)
  shared_dataset.name = 'era'
  shared_dataset.size = 1000
  
  root = fdl.Config(trainer.Trainer)
  root.dataset = shared_dataset
  root.epochs = 5
  
  root.task = fdl.Config(task.Task)
  root.task.task_type = 'classifier'
  
  root.task.model = fdl.Config(model.Model)
  root.task.model.hidden_dim = 64
  
  root.task.model.graph = fdl.Config(graph.Graph)
  root.task.model.graph.dataset = shared_dataset
  root.task.model.graph.nodes = 15
  
  return root